In [1]:
###author : Haim krupkin
#date:04_22_2025
###descriptions:####
#This code is intended to identifiy which mutations and in which proteins are pathogenic  clinvar for cluster 41280 which binds zinc
#

print("hi")

hi


In [2]:
import pandas as pd
import numpy as np
from atom3d.datasets import load_dataset
from tqdm import tqdm
import collections as col
from collapse.utils import pdb_from_fname, deserialize
from collapse.atom_info import aa_to_letter
import matplotlib.pyplot as plt
import seaborn as sns

/oak/stanford/groups/rbaltman/aderry/miniconda3/envs/parse/lib/python3.10/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /oak/stanford/groups/rbaltman/aderry/miniconda3/envs/parse/lib/python3.10/site-packages/torch_sparse/_metis_cuda.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
from collections import defaultdict  # Make sure to import defaultdict


In [4]:
k = 50000
cluster_cts = deserialize(f'data/af2_human_cluster_cts_{k}.pkl')
#cluster_mutations = deserialize(f'data/af2_human_cluster_variants_{k}.pkl')
uniprot_clusters = deserialize(f'data/af2_human_cluster_resids_{k}.pkl')
cluster_refs = deserialize(f'data/af2_human_cluster_variant_refs_{k}.pkl')

In [5]:
clinvar_data_path = "/oak/stanford/groups/rbaltman/hkrupkin/COLLAPSE_WORK/clinvar.txt"
clinvar_data = pd.read_csv(clinvar_data_path, sep=' ')  # Adjust sep if needed

# Display the DataFrame (optional)
print(clinvar_data)

        uniprot_id       label residue_defintions_with_positions
1           P25092     unknown                             A1051
2           Q7Z3K3     unknown                              A568
3           P58012     unknown                               A14
4           P35680  pathogenic                              A373
5           O95409     unknown                               A26
...            ...         ...                               ...
2714070     Q9C0A6      benign                              NANA
2714071     P61764  pathogenic                              NANA
2714072     Q86WG5      benign                              NANA
2714073     P07196      benign                              NANA
2714074     Q9BTV4      benign                              NANA

[2714074 rows x 3 columns]


In [6]:
len(cluster_refs)

23075

In [7]:
rows = []
for cluster_id, amino_acids in cluster_refs.items():
    for amino_acid_id in amino_acids:
        rows.append({'cluster_id': cluster_id, 'amino_acid_id': amino_acid_id})

# Create DataFrame
cluster_and_id = pd.DataFrame(rows)
cluster_and_id[['uniprot_id', 'amino_acid']] = cluster_and_id['amino_acid_id'].str.split('_', expand=True)
#cluster_and_id

In [8]:
df_cluster_mutations_clinvar=pd.merge(clinvar_data,
                                      cluster_and_id,
                                      left_on=['uniprot_id', 'residue_defintions_with_positions'], 
                     right_on=['uniprot_id', 'amino_acid'], 
                     how='outer')#how='inner' #version origanl is inner
                                     
df_cluster_mutations_clinvar

,uniprot_id,label,residue_defintions_with_positions,cluster_id,amino_acid_id,amino_acid
0,P25092,unknown,A1051,NaN,NaN,NaN
1,Q7Z3K3,unknown,A568,NaN,NaN,NaN
2,P58012,unknown,A14,NaN,NaN,NaN
3,P35680,pathogenic,A373,NaN,NaN,NaN
4,P35680,unknown,A373,NaN,NaN,NaN
...,...,...,...,...,...,...
2732927,Q9Y6K1,NaN,NaN,48587.0,Q9Y6K1_P700,P700
2732928,Q9Y6N3,NaN,NaN,32274.0,Q9Y6N3_I104,I104
2732929,Q9Y6N9,NaN,NaN,39700.0,Q9Y6N9_E519,E519
2732930,Q9Y6R1,NaN,NaN,8006.0,Q9Y6R1_A843,A843


In [9]:
###we are gonna identifiy the mutations of intrest and their uniprot and this
#will be used to identify
#the mutations we want to plot
#the cluster of intrest is : 41280
#df_cluster_mutations_clinvar[df_cluster_mutations_clinvar["cluster_id"] == 41280]
#df_cluster_mutations_clinvar[df_cluster_mutations_clinvar["cluster_id"] == 32854]
df_cluster_mutations_clinvar[df_cluster_mutations_clinvar["cluster_id"] == 46115]


,uniprot_id,label,residue_defintions_with_positions,cluster_id,amino_acid_id,amino_acid


In [ ]:
file_name = '/oak/stanford/groups/rbaltman/hkrupkin/COLLAPSE_WORK/df_cluster_mutations_clinvar.csv'

# Write the DataFrame to a CSV file
df_cluster_mutations_clinvar.to_csv(file_name, index=False)

print(f'DataFrame written to {file_name}')